In [1]:
import networkx as nx
import numpy as np
import json 

# --- OpenQAOA Imports ---
from openqaoa import QAOA, create_device
from openqaoa.problems import MaximumCut
from qiskit import qasm3

In [2]:
NUM_QUBITS = 6
GRAPH_SEED = 42
DEPTH = 3 # Number of QAOA layers
MIXER = 'xy' # or 'x'
PARAM_TYPE = 'standard' # or 'extended', 'fourier', etc.
OPTIMIZER_METHOD = 'COBYLA'
OPTIMIZER_MAXITER = 100
OPTIMIZER_TOL = 1e-2
OUTPUT_QASM_FILE = 'parameterized_circuit.qasm3'
OUTPUT_SETTINGS_FILE = 'qaoa_settings.json'

In [ ]:
g = nx.generators.fast_gnp_random_graph(n=NUM_QUBITS, p=0.6, seed=GRAPH_SEED)

# 2. Create QUBO Problem
graphProblem = MaximumCut(g)
qubo_problem = graphProblem.qubo

# 3. Create QAOA object and set parameters
q = QAOA()

intendedDevice = create_device(location="ibmq", name="ibm_brisbane")
q.set_device(intendedDevice)

q.set_circuit_properties(
    p=DEPTH,
    param_type=PARAM_TYPE,
    init_type="ramp", # init_type mainly affects OpenQAOA's optimizer start
    mixer_hamiltonian=MIXER
)

q.set_backend_properties(init_hadamard=True)

q.set_classical_optimizer(method=OPTIMIZER_METHOD, maxiter=OPTIMIZER_MAXITER, tol=OPTIMIZER_TOL) #idk if i really need to do this here but it might be required by compile()

AttributeError: 'QAOA' object has no attribute 'backend'

In [ ]:
parameterized_circuit = q.backend.parametric_circuit #idk why this doesnt work anymore it ran earlier
# Get parameters in the order defined by the circuit/OpenQAOA
# Convert to list of strings (parameter names) for saving
parameter_names = [p.name for p in parameterized_circuit.parameters]

print(f"Exporting circuit to {OUTPUT_QASM_FILE}...")
# Use qiskit.qasm3 exporter available in the environment
qasm3_string = qiskit.qasm3.dumps(parameterized_circuit)
with open(OUTPUT_QASM_FILE, 'w') as f:
    f.write(qasm3_string)
print("Circuit export successful.")

AttributeError: 'QAOA' object has no attribute 'backend'